In [9]:
%load_ext autoreload

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from pycaret.regression import *
from pathlib import Path

In [11]:
%autoreload
import sys
sys.path.append(sys.path[0][0:-4] + 'mikael')
from pipeline import Pipin
pipin = Pipin()

In [12]:

df = pipin.get_combined_datasets()


In [13]:
scaled_data = pipin.scale_targets_in_dataset(df, location="A")

In [14]:
ignore_features = pipin.get_irrelevant_features(df)
categorical_features = pipin.get_categorical_features(df)

In [15]:
scaled_data_0 = scaled_data.sample(frac=1).reset_index(drop=True)

# # I want to make sure the last 30% of the rows have location set to A
# df_a = scaled_data[scaled_data['location:idx'] == 'A']
# df_b = scaled_data[scaled_data['location:idx'] == 'B']
# df_c = scaled_data[scaled_data['location:idx'] == 'C']

# scaled_data = pd.concat([ df_b, df_c,df_a])
scaled_data = scaled_data_0
# I need a to be in the back so that it is tested 
# scaled_data.tail()

In [16]:
s = setup(scaled_data, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, fold_shuffle=False, train_size=0.9)

,Description,Value
0,Session id,6992
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(48032, 53)"
4,Transformed data shape,"(48032, 59)"
5,Transformed train set shape,"(43228, 59)"
6,Transformed test set shape,"(4804, 59)"
7,Ignore features,2
8,Ordinal features,3
9,Numeric features,43


In [17]:
preprocessed_data = s.dataset_transformed
preprocessed_data
print(preprocessed_data.shape)
# print (preprocessed_data.columns)
preprocessed_data.tail(20)

(48032, 59)


,absolute_humidity_2mgm3,air_density_2mkgm3,ceiling_height_aglm,clear_sky_energy_1hJ,clear_sky_radW,cloud_base_aglm,dew_or_rimeidx_0.0,dew_or_rimeidx_1.0,dew_or_rimeidx_-1.0,dew_point_2mK,...,wind_speed_v_10mms,wind_speed_w_1000hPams,is_estimatedidx,day_of_year,hour,month,locationidx_C,locationidx_B,locationidx_A,pv_measurement
48012,3.6,1.292,3017.154917,0.000000e+00,0.000000,1785.329166,1.0,0.0,0.0,269.100006,...,0.5,-0.0,0.0,72.0,5.0,3.0,0.0,0.0,1.0,3.300000
48013,8.1,1.245,1511.800049,2.503170e+04,0.000000,387.299988,1.0,0.0,0.0,281.000000,...,-0.6,0.0,0.0,201.0,21.0,7.0,1.0,0.0,0.0,0.000000
48014,5.8,1.274,5223.000000,1.343220e+06,432.600006,2294.699951,1.0,0.0,0.0,276.000000,...,0.9,-0.0,0.0,106.0,8.0,4.0,0.0,0.0,1.0,1646.699951
48015,10.3,1.240,1577.199951,0.000000e+00,0.000000,842.099976,1.0,0.0,0.0,284.600006,...,-0.2,-0.0,0.0,224.0,3.0,8.0,0.0,0.0,1.0,16.059999
48016,7.2,1.208,8285.500000,5.807049e+05,183.699997,1539.300049,1.0,0.0,0.0,279.200012,...,-0.5,-0.0,0.0,299.0,10.0,10.0,0.0,0.0,1.0,202.179993
48017,4.9,1.281,880.099976,4.244600e+04,26.200001,479.799988,1.0,0.0,0.0,273.500000,...,-0.9,0.0,0.0,325.0,9.0,11.0,0.0,1.0,0.0,148.122177
48018,4.1,1.236,1164.900024,3.785024e+05,132.399994,1164.900024,1.0,0.0,0.0,270.899994,...,2.5,-0.0,0.0,43.0,10.0,2.0,0.0,1.0,0.0,874.846619
48019,4.5,1.287,5836.500000,0.000000e+00,0.000000,702.500000,1.0,0.0,0.0,272.200012,...,0.4,0.0,0.0,118.0,0.0,4.0,0.0,1.0,0.0,0.000000
48020,4.5,1.275,7324.500000,2.208294e+06,586.700012,915.700012,1.0,0.0,0.0,272.299988,...,-3.9,0.0,0.0,108.0,13.0,4.0,0.0,1.0,0.0,2309.780273
48021,8.9,1.224,3249.300049,1.215669e+06,403.100006,558.299988,1.0,0.0,0.0,282.500000,...,-1.3,-0.0,0.0,169.0,6.0,6.0,0.0,0.0,1.0,862.619995


In [18]:
lightgbm = s.create_model('lightgbm')


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,292.0894,245313.4029,495.2912,0.8673,1.3282,2.1804
1,264.0067,203411.4787,451.0116,0.8843,1.3323,2.2452
2,260.9151,203785.1764,451.4257,0.8852,1.3108,1.9042
3,269.9829,211602.8483,460.0031,0.8727,1.2928,1.7485
4,266.3590,209773.1041,458.0099,0.8792,1.3439,2.0617
5,277.6868,227280.1919,476.7391,0.8706,1.3112,2.7462
6,275.4138,225592.0184,474.9653,0.8707,1.3059,1.8475
7,274.4023,225595.6620,474.9691,0.8708,1.3131,2.0094
8,269.3608,214920.7180,463.5954,0.8803,1.3541,2.2391


In [20]:
et = s.create_model('et')


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,229.6022,178513.8141,422.5090,0.9034,0.8139,1.1220
1,204.9435,142852.9394,377.9589,0.9187,0.8065,1.3826
2,202.6480,143758.7409,379.1553,0.9190,0.7623,1.0223
3,211.8986,150632.9325,388.1146,0.9094,0.8079,0.8819
4,210.1416,152458.7353,390.4596,0.9122,0.8106,1.3064
5,221.6587,162451.8840,403.0532,0.9075,0.8415,1.6761
6,216.1682,162822.6808,403.5129,0.9067,0.8087,1.2594
7,219.9061,168406.6630,410.3738,0.9036,0.8424,1.3172
8,217.2231,165871.4642,407.2732,0.9076,0.8481,1.1837


In [21]:
# lightgbm_a_tuned = tune_model(lightgbm, optimize="MAE")

In [27]:
# et = tune_model(et, optimize="MAE")

In [34]:
# tuned = s.tune_model(lightgbm)
# final_a= s.finalize_model(lightgbm_a)
final_et= s.finalize_model(et)
# final_c= c.finalize_model(lightgbm_c)


In [39]:
test_data_a = pipin.get_test_data("A")
test_data_b = pipin.get_combined_test_data(data_sets={"B", "C"})

In [23]:

def get_unique_filename(base_name, folder = "submissions"):
    end = ""
    if folder == "models":
        end = ".pkl"
    
    file_path = Path(folder) / f"{base_name}.csv{end}"
    count = 1
    while file_path.exists():
        file_path = Path(folder) / f"{base_name}_{count}.csv{end}"
        count += 1
    return str(file_path)

In [40]:
res_stack_a = predict_model(final_a, test_data_a)
res_stack_b = predict_model(final_b, test_data_b)
concatinated = pd.concat([res_stack_a, res_stack_b]).reset_index(drop=True)


concatinated

,time,sun_azimuth:d,pressure_100m:hPa,fresh_snow_1h:cm,visibility:m,precip_5min:mm,direct_rad:W,diffuse_rad:W,wind_speed_10m:ms,total_cloud_cover:p,...,dew_or_rime:idx,is_day:idx,is_in_shadow:idx,precip_type_5min:idx,snow_drift:idx,day_of_year,hour,month,location:idx,prediction_label
0,2023-05-01 00:00:00,16.026501,1000.549988,0.0,29907.500000,0.0,0.000000,0.000000,3.950,74.949997,...,0.0,0.0,1.0,0.0,0.0,121,0,5,A,3.108333
1,2023-05-01 01:00:00,30.497250,1000.049988,0.0,29519.074219,0.0,0.000000,0.000000,3.825,77.474998,...,0.0,0.0,1.0,0.0,0.0,121,1,5,A,3.108333
2,2023-05-01 02:00:00,44.517250,999.500000,0.0,31009.125000,0.0,0.000000,0.000000,3.650,88.099998,...,0.0,0.0,1.0,0.0,0.0,121,2,5,A,3.108333
3,2023-05-01 03:00:00,58.083000,998.974976,0.0,34552.500000,0.0,2.100000,9.375000,3.500,68.599998,...,0.0,1.0,0.0,0.0,0.0,121,3,5,A,50.222725
4,2023-05-01 04:00:00,71.341003,998.549988,0.0,35483.875000,0.0,25.450001,47.400002,3.325,66.300003,...,0.0,1.0,0.0,0.0,0.0,121,4,5,A,313.036242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2023-07-03 19:00:00,306.688995,977.575012,0.0,41007.898438,0.0,13.450001,32.200001,2.175,88.175003,...,0.0,1.0,0.0,0.0,0.0,184,19,7,C,62.702372
2156,2023-07-03 20:00:00,319.703003,977.974976,0.0,41315.949219,0.0,2.400000,13.875000,2.200,78.949997,...,0.0,1.0,0.0,0.0,0.0,184,20,7,C,39.681694
2157,2023-07-03 21:00:00,333.037231,978.250000,0.0,41665.898438,0.0,0.000000,1.250000,2.250,80.449997,...,0.0,0.0,1.0,0.0,0.0,184,21,7,C,30.070626
2158,2023-07-03 22:00:00,346.682739,978.650024,0.0,39007.601562,0.0,0.000000,0.000000,1.875,100.000000,...,0.0,0.0,1.0,0.0,0.0,184,22,7,C,38.541425


In [41]:
res_stack = concatinated[["prediction_label"]].rename(columns={"prediction_label": "prediction"}).rename_axis(index="id")

res_stack.to_csv(get_unique_filename('a_model_and_bc_model', 'submissions'), index=True)
# save_model(stacker, model_name=get_unique_filename('stack_gbm_et_rf', "models"))

In [43]:
print("done", res_stack.shape)
pipin.compare_mae(res_stack)


done (2160, 1)


61.693566189292

In [ ]:
# 